In [1]:
from facenet_pytorch import MTCNN,InceptionResnetV1
import cv2
from torch.nn import CosineSimilarity
import os
from PIL import Image
import numpy as np

Testing for a image

In [2]:
# img = cv2.cvtColor(cv2.imread("hand_61.png"), cv2.COLOR_BGR2RGB)
img=Image.open('hand_61.png')
mtcnn = MTCNN(keep_all=True)
img_cropped=mtcnn(img)
# print(img_cropped.shape)
resnet=InceptionResnetV1(pretrained='vggface2').eval()
img_embedding=resnet(img_cropped)
print(img_embedding.shape)
cos=CosineSimilarity()


c:\Users\John\anaconda3\envs\pytorch_gpu\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


torch.Size([1, 512])


Creating embedding for all the faces

In [22]:
names=os.listdir('./faces')
face_embeddings=[]
for i in names:
    img=Image.open(f'./faces/{i}')
    img_cropped=mtcnn(img)
    img_embedding=resnet(img_cropped)
    # print(img_embedding.shape)
    face_embeddings.append(img_embedding)
print(names)
print(face_embeddings[0].shape)

c:\Users\John\anaconda3\envs\pytorch_gpu\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


['deepak.jpg', 'John.jpg']
torch.Size([1, 512])


Real-time processing

In [ ]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    # frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    try:
        boxes = mtcnn.detect(frame)[0]
        for box in boxes:
            box = list(map(int, box))
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), color=(255, 0, 0), thickness=2)
            vids_crop=mtcnn(frame)
            
            # print(vids_crop.shape)
            for vid_crop in vids_crop:
                vid_embedding=resnet(vid_crop.unsqueeze(0))
                res=[]
                for face_embed in face_embeddings:
                    print(face_embed.shape,vid_embedding.shape)
                    similarity=cos(vid_embedding,face_embed)
                    res.append(similarity.detach().numpy())
                res=np.array(res)
                res_ind=res.argmax(0)
    except TypeError:
        pass
    except AttributeError:
        pass
    cv2.imshow('face_detected', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
